# Combining `Portion` and `Pint`

`Portion` module handles intervals and `Pint` handles units. While they can be combined relatively painlessly, the handling of infinity is far from it. 

But before coming to that, we can have a look at how things work when everything is set up.

We have to start with the opticks package import as well as some other useful packages.

In [1]:
# If opticks import fails, try to locate the module
# This can happen building the docs
import os

try:
    import opticks
except ModuleNotFoundError:
    os.chdir(os.path.join("..", ".."))
    os.getcwd()

In [2]:
import warnings

import portion as P

from opticks import Q_, u

warnings.filterwarnings("always")

Units support for portion 'Interval' objects work just fine. Here we initialise two intervals and then compute the "union".

In [3]:
a = P.closed(0 * u.mm, 1 * u.mm)

b = P.closed(1.2 * u.mm, 2.4 * u.mm)

c = a | b

print(c)

[<Quantity(0, 'millimeter')>,<Quantity(1, 'millimeter')>] | [<Quantity(1.2, 'millimeter')>,<Quantity(2.4, 'millimeter')>]


As expected, combining floats with units is a bad idea.

In [4]:
float_interval = P.closed(0, 1)

print(float_interval)
print(b)
try:
    d = float_interval | b
except ValueError:
    print("Can't mix float Intervals with Units!")

[0,1]
[<Quantity(1.2, 'millimeter')>,<Quantity(2.4, 'millimeter')>]
Can't mix float Intervals with Units!


Infinity does not need units, though assigning units to infinity is also possible.

In [5]:
# compare inf to float interval
print(P.inf > float_interval)

# compare inf to Quantity
print(P.inf > 10 * u.mm)

# compare inf to Quantity interval
print(-P.inf < b)

print("****************")

# First way to define Inf with units
ninf = Q_("-inf") * u.Hz
pinf = Q_("+inf Hz")
interval_1 = P.closed(ninf, 10 * u.Hz)
print(interval_1)

# Second way to define Inf with units (may not be fully safe)
interval_2 = P.closed(-P.inf * u.Hz, 10 * u.Hz)
print(interval_2)

# full range Interval
inf = P.open(ninf, pinf)

# check intersection operation
print(interval_1 & inf)

True
True
True
****************
[<Quantity(-inf, 'hertz')>,<Quantity(10, 'hertz')>]
[<Quantity(-inf, 'hertz')>,<Quantity(10, 'hertz')>]
(<Quantity(-inf, 'hertz')>,<Quantity(10, 'hertz')>]


The `IntervalDict` objects are also supported to work with units. The following definition mimicks a band-pass filter that is defined within 20 Hz and there are two bands (0 to 1 Hz and 2 to 3 Hz) where signal passes.

In [6]:
data = P.IntervalDict()
# interval of validity
validity = P.closed(-10 * u.Hz, 10 * u.Hz)
data[validity] = 0
# data proper
first_rng = P.closed(0 * u.Hz, 1 * u.Hz)
data[first_rng] = 1.0
second_rng = P.closed(2 * u.Hz, 3 * u.Hz)
data[second_rng] = 1.0

print(data)

{[<Quantity(-10, 'hertz')>,<Quantity(0, 'hertz')>) | (<Quantity(1, 'hertz')>,<Quantity(2, 'hertz')>) | (<Quantity(3, 'hertz')>,<Quantity(10, 'hertz')>]: 0, [<Quantity(0, 'hertz')>,<Quantity(1, 'hertz')>] | [<Quantity(2, 'hertz')>,<Quantity(3, 'hertz')>]: 1.0}
